# Chatbot con máquina de estado y ontologías

In [1]:
import re

from pprint import pprint
from rivescript import RiveScript
from flask import Flask, request
from dispatcher import ResponseDispatcher

In [8]:
bot = RiveScript()
bot.load_directory('1/')
bot.sort_replies()

while True:
    msg = input('You> ')
    
    # Procesamiento con máquina finita de estados
    
    if msg == '/quit':
        break

    reply = bot.reply("localuser", msg)
    print('Bot> ', reply)

You> /quit


## Implementación de una máquina finita de estados

Se deben de implementar al menos dos clases:

- **`State`**: define lo que es un estado, con un identificador único, guarda información del usuario y métodos para responder un mensaje.
     - Los estados _pueden_ ser clases que hereden de `State` e implementen sus propios métodos de acuerdo a la 
       representación del conocimiento.

```python
# esta línea es falsa y solo se usa a manera de ejemplo
import facebook_api

class State:

    def __init__(self, id_, user_id, **kwargs):
        self.id = id_
        self.user_id = user_id
        self.next_states = []
        
    def respond(self, msg):
        facebook_api.respond(msg, self.user_id)
        
        
class Estado_1(State):

    def __init__(self,  id_, user_id, global_frame, **kwargs):
        State.__init__(self, id_, user_id)
        self.global_frame = global_frame
        self.username = global_frame['first_name']
        self.lastname = global_frame['last_name']
    
    def run(self):
        self.action1()
        self.action2()
        # ejecuta más estados
        # .
        # .
        # .
        # los estados siguientes deben de ir YA instanciados
        self.next_states = [Estado_2(self.id, self.user_id, self.global_frame),
                            Estado_4(self.id, self.user_id, self.global_frame)]
        
    def action1(self):
        msg = 'Je pense donc je suis'
        self.respond(msg)
        
    def get_next_state(self, user_input):
        # Lista de if-else con expresiones regulares
        pass
```

       
- **`FSM`**: la clase que maneja el control de la máquina de estados

```python
class FSM:
    
    def __init__(self, current_state):
        self.current_state = current_state
        
    def run(self):
        self.current_state.run()
        
    def transition(self, user_input):
        self.current_state = self.current_state.get_next_state(user_input)
```

## Servidor con conexión a Facebook Messenger

In [2]:
app = Flask(__name__)
DISPATCHER = ResponseDispatcher()

In [3]:
@app.route('/', methods=['GET'])
def verify():
    """
    when the endpoint is registered as a webhook, it must echo back
    the 'hub.challenge' value it receives in the query arguments
    """
    if request.args.get('hub.mode') == 'subscribe' and request.args.get('hub.challenge'):
        if not request.args.get('hub.verify_token') == os.environ['VERIFY_TOKEN']:
            return 'Verification token mismatch', 403
        return request.args['hub.challenge'], 200

    print('Hello world')
    return 'Hello world', 200

@app.route('/', methods=['POST'])
def webhook():
    """
    Endpoint for processing incoming messaging events
    """
    global DISPATCHER
    data = request.get_json()
    pprint(data)
    DISPATCHER.dispatch(data)
    return 'ok', 200

In [7]:
%tb
app.run(debug=True, host='0.0.0.0', port=5500)

SystemExit: 1

OSError: [Errno 48] Address already in use

In [48]:
def gen():
    lista = [1,2,3,4]
    for e in lista:
        yield e
        
g = gen()

In [53]:
next(g)

StopIteration: 